In [1]:
#from sagemaker.local import LocalSession
import sagemaker
from sagemaker import get_execution_role

#sagemaker_session = sagemaker.Session()
sagemaker_session = sagemaker.LocalSession()
#sagemaker_session.config={'local': {'local_code': True}}


# Get a SageMaker-compatible role used by this Notebook Instance.
#role = get_execution_role()
role = "arn:aws:iam::961558279552:role/ravindra"
#train_input = 's3://1814-tigerassign1/B/housing.csv'
# S3 prefix
bucket = sagemaker_session.default_bucket()
prefix = 'SkLinearLearner-pipeline'

In [2]:

#WORK_DIRECTORY = 'housing_data'
WORK_DIRECTORY = '.'

train_input = sagemaker_session.upload_data(
    path='{}/{}'.format(WORK_DIRECTORY, 'housing.csv'), 
    bucket=bucket,
    key_prefix='{}/{}'.format(prefix, 'train'))

# Create SageMaker Scikit Estimator 


In [3]:
from sagemaker.sklearn.estimator import SKLearn

FRAMEWORK_VERSION = "0.23-1"
script_path = './sklearn_pipe1.py'

sklearn_preprocessor = SKLearn(
    entry_point=script_path,
    role=role,
    framework_version=FRAMEWORK_VERSION,
    instance_type="ml.m4.xlarge",
    #instance_type = "local",
    sagemaker_session=sagemaker_session)


In [4]:
train_input

's3://sagemaker-ap-south-1-961558279552/SkLinearLearner-pipeline/train/housing.csv'

In [5]:
sklearn_preprocessor.fit({'train': train_input})

Creating jsjtnd6tx0-algo-1-0bnyn ... 
Creating jsjtnd6tx0-algo-1-0bnyn ... done
Attaching to jsjtnd6tx0-algo-1-0bnyn
jsjtnd6tx0-algo-1-0bnyn | 2021-06-14 12:53:59,934 sagemaker-containers INFO     Imported framework sagemaker_sklearn_container.training
jsjtnd6tx0-algo-1-0bnyn | 2021-06-14 12:53:59,937 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
jsjtnd6tx0-algo-1-0bnyn | 2021-06-14 12:53:59,949 sagemaker_sklearn_container.training INFO     Invoking user training script.
jsjtnd6tx0-algo-1-0bnyn | 2021-06-14 12:54:00,506 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
jsjtnd6tx0-algo-1-0bnyn | 2021-06-14 12:54:00,533 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
jsjtnd6tx0-algo-1-0bnyn | 2021-06-14 12:54:00,553 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
jsjtnd6tx0-algo-1-0bnyn | 2021-06-14 12:54:00,567 sagemaker-training-toolkit INFO     In

RuntimeError: Failed to run: ['docker-compose', '-f', '/tmp/tmpua_dtstm/docker-compose.yaml', 'up', '--build', '--abort-on-container-exit'], Process exited with code: 1

# Batch transform our training data 

In [ ]:
# Define a SKLearn Transformer from the trained SKLearn Estimator
transformer = sklearn_preprocessor.transformer(
    instance_count=1, 
    instance_type='ml.m4.xlarge',
    #instance_type='local',
    assemble_with = 'Line',
    accept = 'text/csv')

In [ ]:

# Preprocess training input
transformer.transform(train_input, content_type="text/csv")
print("Waiting for transform job: " + transformer.latest_transform_job.job_name)
transformer.wait()
preprocessed_train = transformer.output_path

# Serial Inference Pipeline with Scikit preprocessor and Linear Learner


In [ ]:
from sagemaker.model import Model
from sagemaker.pipeline import PipelineModel
import boto3
from time import gmtime, strftime

timestamp_prefix = strftime("%Y-%m-%d-%H-%M-%S", gmtime())

scikit_learn_inference_model = sklearn_preprocessor.create_model()

model_name = 'inference-pipeline-' + timestamp_prefix
endpoint_name = 'inference-pipeline-ep-' + timestamp_prefix


scikit_learn_inference_model.deploy(initial_instance_count=1, instance_type='ml.m4.xlarge', endpoint_name=endpoint_name)

# Make a request to our pipeline endpoint


In [ ]:

from sagemaker.predictor import Predictor
from sagemaker.serializers import CSVSerializer

payload = '-122.23,37.88,41.0,880.0,129.0,322.0,126.0,8.3252,NEAR BAY,452600.0'
actual_rings = 10
predictor = Predictor(
    endpoint_name=endpoint_name,
    sagemaker_session=sagemaker_session,
    serializer=CSVSerializer())

print(predictor.predict(payload))

# Delete Endpoint

In [ ]:

sm_client = sagemaker_session.boto_session.client('sagemaker')
sm_client.delete_endpoint(EndpointName=endpoint_name)